In [1]:
import json
from sentence_transformers import SentenceTransformer, util
from summarizer.sbert import SBertSummarizer
from transformers import BertTokenizer,AlbertTokenizer,AutoTokenizer, AlbertForQuestionAnswering, AutoModelForQuestionAnswering ,BertForQuestionAnswering

from transformers import BertTokenizer
import torch

from transformers import pipeline

In [2]:
data_size = 10
search_text_size = 128

In [3]:
articles = []
file = open('data/val.txt', encoding='utf-8', mode='r')
for i in range(data_size):
    line = file.readline()
    art = json.loads(line)
    abstract = " ".join([l.replace('<S>', '').replace('</S>', '') for l in art['abstract_text']])
    passage = " ".join(art['article_text'])
    articles.append([abstract, passage])
file.close()

In [4]:
model = SBertSummarizer('paraphrase-MiniLM-L6-v2')
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
# print(articles[0][1])
# print("================================")
abses = []
arts = []
for abs, art in articles:
    if len(abs.split()) > 256:
        abs = model(abs, min_length=search_text_size)
    if len(art.split()) > 256:
        art = model(art, min_length=search_text_size)
    abses.append(abs)
    arts.append(art)

arts_embeddings = encoder.encode(arts)
abs_embeddings = encoder.encode(abses)

In [6]:
abs_embeddings.shape
arts_embeddings.shape

(10, 384)

In [29]:
question = "who decides whether stroke status is correct?"
question_embedding = encoder.encode(question, convert_to_tensor=True)
hits = util.semantic_search(question_embedding, arts_embeddings, top_k=5)
reference_text = articles[hits[0][0]['corpus_id']][1]

In [30]:
hits

[[{'corpus_id': 1, 'score': 0.5580443739891052},
  {'corpus_id': 0, 'score': 0.36922186613082886},
  {'corpus_id': 2, 'score': 0.2462112158536911},
  {'corpus_id': 4, 'score': 0.2442772388458252},
  {'corpus_id': 5, 'score': 0.19890615344047546}]]

In [31]:
articles[1][0]

' backgroundthe questionnaire for verifying stroke - free status ( qvsfs ) has been validated in western populations as a method for verifying stroke - free status in participants of clinical , epidemiological and genetic studies .   this instrument has not been validated in low - income settings where populations have limited knowledge of stroke symptoms and literacy levels are low.objectiveto simultaneously validate the 8-item qvsfs in 3 languages spoken in west africa ( yoruba , hausa and akan ) for ascertainment of stroke - free status of control subjects in siren.methodsusing a cross - sectional study design , 100 participants each from the 3 linguistic groups will be consecutively recruited from neurology and general medicine clinics of 5 tertiary referral hospitals in nigeria and ghana .   ascertainment of stroke status will be determined by neurologists using structured neurological examination , review of case records and neuro - imaging ( gold standard ) .   the relative perf

In [32]:
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

In [33]:
window_size = 256
stride = 128
reference_text = articles[hits[0][0]['corpus_id']][1]
sub = []
words = reference_text.split()
for i in range(0, len(words) - stride, stride):
    sub_text = " ".join(words[i:i+window_size])
    sub.append(sub_text)

sub_embs = encoder.encode(sub)
sub_hits = util.semantic_search(question_embedding, sub_embs, top_k=5)
for h in sub_hits[0]:
    print(h)


{'corpus_id': 26, 'score': 0.6570421457290649}
{'corpus_id': 0, 'score': 0.6569384932518005}
{'corpus_id': 1, 'score': 0.6192595958709717}
{'corpus_id': 17, 'score': 0.6186014413833618}
{'corpus_id': 8, 'score': 0.6182590126991272}


In [34]:
sub_reference_text = sub[sub_hits[0][0]['corpus_id']]
result = question_answerer(question=question, context=sub_reference_text)
print(result)

{'score': 0.9201184511184692, 'start': 484, 'end': 496, 'answer': 'neurologists'}
